In [ ]:
!sudo add-apt-repository -y ppa:djcj/hybrid
!sudo apt-get update
!sudo apt-get install ffmpeg -y

#!conda remove ffmpeg
#!conda install -c joemarct ffmpeg 
!pip install ffmpeg-python

In [10]:
!ls ../input_video

play_dead.mp4


In [1]:
%%time
import ffmpeg
import moviepy.editor as mpe
import os



def crop_video(input_video_path, ith_video, jth_horizontal_partition, 
        kth_vertical_partition, num_horizontal_partitions, num_vertical_partitions):    
    video_dir = os.path.dirname(input_video_path)
    video_filename = os.path.basename(input_video_path)
    
    video_width, video_height = mpe.VideoFileClip(input_video_path).size
    y_delta = int(video_height / num_horizontal_partitions)
    x_delta = int(video_width / num_vertical_partitions)
    x1 = kth_vertical_partition * x_delta
    y1 = jth_horizontal_partition * y_delta

    output_video = '{}_{}'.format(ith_video, video_filename)
    output_video_path = os.path.join(video_dir, output_video)
    if os.path.isfile(output_video_path):
        os.remove(output_video_path)
    (ffmpeg
        .input(input_video_path)
        .crop(x1, y1, x_delta, y_delta) # this requires left upper corner and deltas
        .output(output_video_path)
        .run()
    )

if __name__ == "__main__":
    num_horizontal_partitions = 2 ### define here
    num_vertical_partitions = 2 ### define here
    input_video_path = '../input_video/play_dead.mp4' ### define here

    ith_video = 0
    for jth_horizontal_partition in range(num_horizontal_partitions):
        for kth_vertical_partition in range(num_vertical_partitions):
            crop_video(input_video_path, ith_video, 
                       jth_horizontal_partition, kth_vertical_partition, 
                       num_horizontal_partitions, num_vertical_partitions)
            ith_video += 1

CPU times: user 456 ms, sys: 148 ms, total: 604 ms
Wall time: 3.57 s


In [1]:
%%time
import moviepy.editor as mpe
import numpy as np
import os

## need to add audio
def stitch_clips(input_video_path, num_horizontal_partitions, num_vertical_partitions, border_margin=10): # add 10px contour
    @np.vectorize
    def get_clips(video_dir, video_filename, clip_index, border_margin):
        clip_path = os.path.join(video_dir, '{}_{}'.format(clip_index, video_filename))
        return mpe.VideoFileClip(clip_path).margin(border_margin)

    video_dir = os.path.dirname(input_video_path)
    video_filename = os.path.basename(input_video_path)
    clip_index = np.arange(num_horizontal_partitions * num_vertical_partitions).reshape(num_horizontal_partitions, num_vertical_partitions)
    all_clips = get_clips(video_dir, video_filename, clip_index, border_margin)
    final_clip = mpe.clips_array(all_clips)
    #audio_background = mpe.AudioFileClip(input_video_path)
    #final_clip = final_clip.set_audio(audio_background)
    output_video_path = os.path.join(video_dir, "stitched_" + video_filename)
    final_clip.write_videofile(output_video_path)

    
input_video_path = '../input_video/play_dead.mp4' ## define here
num_horizontal_partitions = 2 ## define here
num_vertical_partitions = 2 ## define here
stitch_clips(input_video_path, num_horizontal_partitions, num_vertical_partitions)

[MoviePy] >>>> Building video ../input_video/stitched_play_dead.mp4
[MoviePy] Writing video ../input_video/stitched_play_dead.mp4


100%|██████████| 158/158 [00:02<00:00, 63.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../input_video/stitched_play_dead.mp4 

CPU times: user 1.95 s, sys: 264 ms, total: 2.21 s
Wall time: 3.83 s


### DEPRECATED since slower

In [1]:
%%time
# ideally can run faster with more CPUs or use ffmpeg
from moviepy.editor import VideoFileClip
import itertools
import os
from multiprocessing import Pool


def crop_video_wrapper(tup):
    return crop_video(*tup)

def crop_video(input_video_filename, ith_video, jth_horizontal_partition, 
               kth_vertical_partition, num_horizontal_partitions, num_vertical_partitions):
    clip = VideoFileClip(input_video_filename)
    video_width, video_height = clip.size
    y_delta = int(video_height / num_horizontal_partitions)
    x_delta = int(video_width / num_vertical_partitions)

    x1 = kth_vertical_partition * x_delta
    y1 = jth_horizontal_partition * y_delta
    x2 = (kth_vertical_partition + 1) * x_delta
    y2 = (jth_horizontal_partition + 1) * y_delta
    cropped_clip = clip.crop(x1, y1, x2, y2)
    output_video = 'moviepy_output_{}.mp4'.format(ith_video)
    if os.path.isfile(output_video):
        os.remove(output_video)
    cropped_clip.write_videofile(output_video, audio=False)


if __name__ == '__main__':
    num_horizontal_partitions = 2 ### define here
    num_vertical_partitions = 3 ### define here
    input_video_filename = '1_minute_720p_video.mp4' ### define here
    num_processes = 2 ### define here
    
    grid = []
    ith_video = 0
    for jth_horizontal_partition in range(num_horizontal_partitions):
        for kth_vertical_partition in range(num_vertical_partitions):
            grid.append((input_video_filename, ith_video, jth_horizontal_partition, 
                kth_vertical_partition, num_horizontal_partitions, num_vertical_partitions))
            ith_video += 1
        
    pool = Pool(num_processes)
    pool.map(crop_video_wrapper, grid)
    pool.close()
    """
    for ith_video, (jth_horizontal_partition, kth_vertical_partition) in enumerate(
    itertools.product(range(num_horizontal_partitions), range(num_vertical_partitions))):
    crop_video(input_video_filename, ith_video, jth_horizontal_partition, kth_vertical_partition)
    """;

[MoviePy] >>>> Building video moviepy_output_1.mp4
[MoviePy] >>>> Building video moviepy_output_0.mp4
[MoviePy] Writing video moviepy_output_0.mp4
[MoviePy] Writing video moviepy_output_1.mp4


100%|█████████▉| 1799/1802 [00:34<00:00, 51.82it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: moviepy_output_0.mp4 



100%|██████████| 1802/1802 [00:34<00:00, 51.86it/s]


[MoviePy] >>>> Building video moviepy_output_2.mp4
[MoviePy] Writing video moviepy_output_2.mp4


...

[MoviePy] Done.
[MoviePy] >>>> Video ready: moviepy_output_2.mp4 

[MoviePy] Done.
[MoviePy] >>>> Video ready: moviepy_output_3.mp4 

[MoviePy] >>>> Building video moviepy_output_4.mp4
[MoviePy] Writing video moviepy_output_4.mp4


  1%|          | 11/1802 [00:00<00:16, 108.50it/s]

[MoviePy] >>>> Building video moviepy_output_5.mp4
[MoviePy] Writing video moviepy_output_5.mp4


100%|██████████| 1802/1802 [00:35<00:00, 50.93it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: moviepy_output_4.mp4 

[MoviePy] Done.
[MoviePy] >>>> Video ready: moviepy_output_5.mp4 

CPU times: user 1.34 s, sys: 560 ms, total: 1.9 s
Wall time: 1min 46s


In [ ]:
import moviepy.editor as mpe

my_clip = mpe.VideoFileClip('my_stack.mp4')
audio_background = mpe.AudioFileClip(input_video_path)
final_clip = my_clip.set_audio(audio_background)
final_clip.write_videofile('stitched_video.mp4')

[MoviePy] >>>> Building video my_stack_with_audio.mp4
[MoviePy] Writing audio in my_stack_with_audioTEMP_MPY_wvf_snd.mp3


100%|██████████| 1325/1325 [00:01<00:00, 1324.92it/s]


[MoviePy] Done.
[MoviePy] Writing video my_stack_with_audio.mp4


 14%|█▍        | 251/1804 [00:07<00:44, 35.19it/s]